In [2]:
# pip install cellregmap

In [3]:
import cellregmap as CellRegMap

In [4]:
import pandas as pd

In [5]:
import numpy as np

In [5]:
import matplotlib as plt

In [6]:
import hail as hl

In [7]:
mt = hl.read_matrix_table('gs://cpg-tob-wgs-test/tob_wgs_vep/v0/vep105_GRCh38.mt')

Initializing Hail with default parameters...


2022-03-14 23:58:48 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-03-14 23:58:50 WARN  Hail:43 - This Hail JAR was compiled for Spark 3.1.1, running with Spark 3.1.2.
  Compatibility is not guaranteed.


Running on Apache Spark version 3.1.2
SparkUI available at http://annas-notebook.australia-southeast1-a.c.notebooks-314505.internal:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.73-c6f6f09cec08
LOGGING: writing to /home/jupyter/hail-20220314-2358-0.2.73-c6f6f09cec08.log


In [30]:
# mt.describe()

In [9]:
# going through this tutorial: https://hail.is/docs/0.2/tutorials/01-genome-wide-association-study.html
# mt is a "MatrixTable" object type, specific to hail (I think?)

In [27]:
# dimension (equivalent to .shape)
# 77,899,783 variants
# 1,061 individuals
mt.count() # (rows, cols)

(77899783, 1061)

In [9]:
# rows are genomic variants,
# if nothing is specified only the "row key fields" are displayed, i.e., "locus" and "alleles"
mt.rows().select().show(5)

,
locus,alleles
locus<GRCh38>,array<str>
chr1:10011,"[""C"",""T""]"
chr1:10020,"[""A"",""C""]"
chr1:10027,"[""A"",""G""]"
chr1:10109,"[""AACCCT"",""A""]"
chr1:10114,"[""T"",""C""]"


In [ ]:
# mt.rows().describe()

In [18]:
# columns are samples (individuals) - "CPGXX" are sample names
# mt.col.describe()
mt.s.show(5)

""
s
str
"""CPG18"""
"""CPG26"""
"""CPG34"""
"""CPG42"""
"""CPG59"""


In [40]:
tob_wgs = hl.experimental.densify(mt)

In [42]:
tob_wgs = hl.variant_qc(tob_wgs)

In [43]:
tob_wgs = tob_wgs.filter_rows(tob_wgs.variant_qc.AF[0] < 1)

In [ ]:
tob_wgs.count()

In [ ]:
snp_maf_05 = tob_wgs.aggregate_rows(hl.agg.count_where(tob_wgs.variant_qc.AF[1] > 0.05))

In [ ]:
print(f'Variant MAF > 0.05 = {snp_maf_05}')

In [ ]:
snp_maf_01 = tob_wgs.aggregate_rows(hl.agg.count_where(tob_wgs.variant_qc.AF[1] > 0.01))

In [ ]:
print(f'Variant MAF > 0.01 = {snp_maf_01}')

In [ ]:
hl.init(default_reference='GRCh38')

    tob_wgs = hl.read_matrix_table(TOB_WGS)
    tob_wgs = hl.experimental.densify(tob_wgs)
    tob_wgs = hl.variant_qc(tob_wgs)
    # get MAF > 0.05
    tob_wgs = tob_wgs.filter_rows(tob_wgs.variant_qc.AF[0] < 1)
    snp_maf_05 = tob_wgs.aggregate_rows(
        hl.agg.count_where(tob_wgs.variant_qc.AF[1] > 0.05)
    )
    print(f'Variant MAF > 0.05 = {snp_maf_05}')